In [1]:
#from google.colab import drive

import os
import sys
import re
import string
import nltk
import pandas as pd
import numpy as np
import keras

from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from sklearn.model_selection import train_test_split
from keras.layers import LSTM, Dropout, Dense, Bidirectional,  Flatten, Input, GRU
from keras.layers.embeddings import Embedding

from unidecode import unidecode

from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

from gensim.models import KeyedVectors
from gensim.models import Word2Vec

nltk.download('stopwords')
stopwords=stopwords.words('english')

T = TweetTokenizer()
EMBEDDING_FILE = '../GoogleNews-vectors-negative300.bin'

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/beuvry_j/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def value_sentiment(x):
  if x == "neutral":
    return 0
  if x == "positive":
    return 1
  if x == "negative":
    return -1

In [3]:
def onehot (tweets,length):
  onh = []
  for tweet in tweets:
      onh_seq = np.zeros(length) 
      for i in tweet:
        onh_seq[i -1] = 1
      onh.append(onh_seq)
  return onh

In [4]:
#replace emojy
emojis = {
    u":‑\)":"☺️",
    u":\)":"☺️",
    u":-\]":"☺️",
    u":\]":"☺️",
    u":-3":"☺️",
    u":3":"☺️",
    u":->":"☺️",
    u":>":"☺️",
    u"8-\)":"☺️",
    u":o\)":"☺️",
    u":-\}":"☺️",
    u":\}":"☺️",
    u":-\)":"☺️",
    u":c\)":"☺️",
    u":\^\)":"☺️",
    u"=\]":"☺️",
    u"=\)":"☺️",
    u":‑D":"😃",
    u":D":"😃",
    u"8‑D":"😃",
    u"8D":"😃",
    u"X‑D":"😃",
    u"XD":"😃",
    u"=D":"😃",
    u"=3":"😃",
    u"B\^D":"😃",
    u":-\)\)":"😃",
    u":‑\(":"☹️",
    u":-\(":"☹️",
    u":\(":"☹️",
    u":‑c":"☹️",
    u":c":"☹️",
    u":‑<":"☹️",
    u":<":"☹️",
    u":‑\[":"☹️",
    u":\[":"☹️",
    u":-\|\|":"☹️",
    u">:\[":"☹️",
    u":\{":"☹️",
    u":@":"☹️",
    u">:\(":"☹️",
    u":'‑\(":"😭",
    u":'\(":"😭",
    u":'‑\)":"😃",
    u":'\)":"😃",
    u"D‑':":"😨",
    u"D:<":"😨",
    u"D:":"😧",
    u"D8":"😧",
    u"D;":"😧",
    u"D=":"😧",
    u"DX":"😧",
    u":‑O":"😮",
    u":O":"😮",
    u":‑o":"😮",
    u":o":"😮",
    u":-0":"😮",
    u"8‑0":"😮",
    u">:O":"😮",
    u":-\*":"😗",
    u":\*":"😗",
    u":X":"😗",
    u";‑\)":"😉",
    u";\)":"😉",
    u"\*-\)":"😉",
    u"\*\)":"😉",
    u";‑\]":"😉",
    u";\]":"😉",
    u";\^\)":"😉",
    u":‑,":"😉",
    u";D":"😉",
    u":‑P":"😛",
    u":P":"😛",
    u"X‑P":"😛",
    u"XP":"😛",
    u":‑Þ":"😛",
    u":Þ":"😛",
    u":b":"😛",
    u"d:":"😛",
    u"=p":"😛",
    u">:P":"😛",
    u":‑/":"😕",
    u":/":"😕",
    u":-[.]":"😕",
    u">:[(\\\)]":"😕",
    u">:/":"😕",
    u":[(\\\)]":"😕",
    u"=/":"😕",
    u"=[(\\\)]":"😕",
    u":L":"😕",
    u"=L":"😕",
    u":S":"😕",
    u":‑\|":"😐",
    u":\|":"😐",
    u":$":"😳",
    u":‑x":"🤐",
    u":x":"🤐",
    u":‑#":"🤐",
    u":#":"🤐",
    u":‑&":"🤐",
    u":&":"🤐",
    u"O:‑\)":"😇",
    u"O:\)":"😇",
    u"0:‑3":"😇",
    u"0:3":"😇",
    u"0:‑\)":"😇",
    u"0:\)":"😇",
    u":‑b":"😛",
    u"0;\^\)":"😇",
    u">:‑\)":"😈",
    u">:\)":"😈",
    u"\}:‑\)":"😈",
    u"\}:\)":"😈",
    u"3:‑\)":"😈",
    u"3:\)":"😈",
    u">;\)":"😈",
    u"\|;‑\)":"😎",
    u"\|‑O":"😏",
    u":‑J":"😏",
    u"%‑\)":"😵",
    u"%\)":"😵",
    u":-###..":"🤒",
    u":###..":"🤒",
    u"\(>_<\)":"😣",
    u"\(>_<\)>":"😣",
    u"\(';'\)":"👶",
    u"\(\^\^>``":"😓",
    u"\(\^_\^;\)":"😓",
    u"\(-_-;\)":"😓",
    u"\(~_~;\) \(・\.・;\)":"😓",
    u"\(-_-\)zzz":"😴",
    u"\(\^_-\)":"😉",
    u"\(\(\+_\+\)\)":"😕",
    u"\(\+o\+\)":"😕",
    u"\^_\^":"😃",
    u"\(\^_\^\)/":"😃",
    u"\(\^O\^\)／":"😃",
    u"\(\^o\^\)／":"😃",
    u"\(__\)":"🙇",
    u"_\(\._\.\)_":"🙇",
    u"<\(_ _\)>":"🙇",
    u"<m\(__\)m>":"🙇",
    u"m\(__\)m":"🙇",
    u"m\(_ _\)m":"🙇",
    u"\('_'\)":"😭",
    u"\(/_;\)":"😭",
    u"\(T_T\) \(;_;\)":"😭",
    u"\(;_;":"😭",
    u"\(;_:\)":"😭",
    u"\(;O;\)":"😭",
    u"\(:_;\)":"😭",
    u"\(ToT\)":"😭",
    u";_;":"😭",
    u";-;":"😭",
    u";n;":"😭",
    u";;":"😭",
    u"Q\.Q":"😭",
    u"T\.T":"😭",
    u"QQ":"😭",
    u"Q_Q":"😭",
    u"\(-\.-\)":"😞",
    u"\(-_-\)":"😞",
    u"\(一一\)":"😞",
    u"\(；一_一\)":"😞",
    u"\(=_=\)":"😩",
    u"\(=\^\·\^=\)":"😺",
    u"\(=\^\·\·\^=\)":"😺",
    u"=_\^=	":"😺",
    u"\(\.\.\)":"😔",
    u"\(\._\.\)":"😔",
    u"\(\・\・?":"😕",
    u"\(?_?\)":"😕",
    u">\^_\^<":"😃",
    u"<\^!\^>":"😃",
    u"\^/\^":"😃",
    u"\（\*\^_\^\*）" :"😃",
    u"\(\^<\^\) \(\^\.\^\)":"😃",
    u"\(^\^\)":"😃",
    u"\(\^\.\^\)":"😃",
    u"\(\^_\^\.\)":"😃",
    u"\(\^_\^\)":"😃",
    u"\(\^\^\)":"😃",
    u"\(\^J\^\)":"😃",
    u"\(\*\^\.\^\*\)":"😃",
    u"\(\^—\^\）":"😃",
    u"\(#\^\.\^#\)":"😃",
    u"\（\^—\^\）":"👋",
    u"\(;_;\)/~~~":"👋",
    u"\(\^\.\^\)/~~~":"👋",
    u"\(T_T\)/~~~":"👋",
    u"\(ToT\)/~~~":"👋",
    u"\(\*\^0\^\*\)":"😍",
    u"\(\*_\*\)":"😍",
    u"\(\*_\*;":"😍",
    u"\(\+_\+\) \(@_@\)":"😍",
    u"\(\*\^\^\)v":"😂",
    u"\(\^_\^\)v":"😂",
    u'\(-"-\)':"😓",
    u"\(ーー;\)":"😓",
    u"\(\^0_0\^\)":"😎",
    u"\(\＾ｖ\＾\)":"😀",
    u"\(\＾ｕ\＾\)":"😀",
    u"\(\^\)o\(\^\)":"😀",
    u"\(\^O\^\)":"😀",
    u"\(\^o\^\)":"😀",
    u"\)\^o\^\(":"😀",
    u":O o_O":"😮",
    u"o_0":"😮",
    u"o\.O":"😮",
    u"\(o\.o\)":"😮",
    u"oO":"😮",
    u"\(\*￣m￣\)":"😠",
    u":‑)":"☺️",
    u":)":"☺️",
    u":-]":"☺️",
    u":]":"☺️",
    u":-3":"☺️",
    u":3":"☺️",
    u":->":"☺️",
    u":>":"☺️",
    u"8-)":"☺️",
    u":o)":"☺️",
    u":-}":"☺️",
    u":}":"☺️",
    u":-)":"☺️",
    u":c)":"☺️",
    u":^)":"☺️",
    u"=]":"☺️",
    u"=)":"☺️",
    u":‑D":"😃",
    u":D":"😃",
    u"8‑D":"😃",
    u"8D":"😃",
    u"X‑D":"😃",
    u"XD":"😃",
    u"=D":"😃",
    u"=3":"😃",
    u"B^D":"😃",
    u":-))":"😃",
    u":-(":"☹️",
    u":‑(":"☹️",
    u":(":"☹️",
    u":‑c":"☹️",
    u":c":"☹️",
    u":‑<":"☹️",
    u":<":"☹️",
    u":‑[":"☹️",
    u":[":"☹️",
    u":-||":"☹️",
    u">:[":"☹️",
    u":{":"☹️",
    u":@":"☹️",
    u">:(":"☹️",
    u":'‑(":"😭",
    u":'(":"😭",
    u":'‑)":"😃",
    u":')":"😃",
    u"D‑':":"😧",
    u"D:<":"😨",
    u"D:":"😧",
    u"D8":"😧",
    u"D;":"😧",
    u"D=":"😧",
    u"DX":"😧",
    u":‑O":"😮",
    u":O":"😮",
    u":‑o":"😮",
    u":o":"😮",
    u":-0":"😮",
    u"8‑0":"😮",
    u">:O":"😮",
    u":-*":"😗",
    u":*":"😗",
    u":X":"😗",
    u";‑)":"😉",
    u";)":"😉",
    u"*-)":"😉",
    u"*)":"😉",
    u";‑]":"😉",
    u";]":"😉",
    u";^)":"😉",
    u":‑,":"😉",
    u";D":"😉",
    u":‑P":"😛",
    u":P":"😛",
    u"X‑P":"😛",
    u"XP":"😛",
    u":‑Þ":"😛",
    u":Þ":"😛",
    u":b":"😛",
    u"d:":"😛",
    u"=p":"😛",
    u">:P":"😛",
    u":‑/":"😕",
    u":/":"😕",
    u":-[.]":"😕",
    u">:[(\)]":"😕",
    u">:/":"😕",
    u":[(\)]":"😕",
    u"=/":"😕",
    u"=[(\)]":"😕",
    u":L":"😕",
    u"=L":"😕",
    u":S":"😕",
    u":‑|":"😐",
    u":|":"😐",
    u":$":"😳",
    u":‑x":"🤐",
    u":x":"🤐",
    u":‑#":"🤐",
    u":#":"🤐",
    u":‑&":"🤐",
    u":&":"🤐",
    u"O:‑)":"😇",
    u"O:)":"😇",
    u"0:‑3":"😇",
    u"0:3":"😇",
    u"0:‑)":"😇",
    u"0:)":"😇",
    u":‑b":"😛",
    u"0;^)":"😇",
    u">:‑)":"😈",
    u">:)":"😈",
    u"}:‑)":"😈",
    u"}:)":"😈",
    u"3:‑)":"😈",
    u"3:)":"😈",
    u">;)":"😈",
    u"|;‑)":"😎",
    u"|‑O":"😏",
    u":‑J":"😏",
    u"%‑)":"😵",
    u"%)":"😵",
    u":-###..":"🤒",
    u":###..":"🤒",
    u"(>_<)":"😣",
    u"(>_<)>":"😣",
    u"(';')":"Baby",
    u"(^^>``":"😓",
    u"(^_^;)":"😓",
    u"(-_-;)":"😓",
    u"(~_~;) (・.・;)":"😓",
    u"(-_-)zzz":"😴",
    u"(^_-)":"😉",
    u"((+_+))":"😕",
    u"(+o+)":"😕",
    u"^_^":"😃",
    u"(^_^)/":"😃",
    u"(^O^)／":"😃",
    u"(^o^)／":"😃",
    u"(__)":"🙇",
    u"_(._.)_":"🙇",
    u"<(_ _)>":"🙇",
    u"<m(__)m>":"🙇",
    u"m(__)m":"🙇",
    u"m(_ _)m":"🙇",
    u"('_')":"😭",
    u"(/_;)":"😭",
    u"(T_T) (;_;)":"😭",
    u"(;_;":"😭",
    u"(;_:)":"😭",
    u"(;O;)":"😭",
    u"(:_;)":"😭",
    u"(ToT)":"😭",
    u";_;":"😭",
    u";-;":"😭",
    u";n;":"😭",
    u";;":"😭",
    u"Q.Q":"😭",
    u"T.T":"😭",
    u"QQ":"😭",
    u"Q_Q":"😭",
    u"(-.-)":"😞",
    u"(-_-)":"😞",
    u"(一一)":"😞",
    u"(；一_一)":"😞",
    u"(=_=)":"😩",
    u"(=^·^=)":"😺",
    u"(=^··^=)":"😺",
    u"=_^= ":"😺",
    u"(..)":"😔",
    u"(._.)":"😔",
    u"(・・?":"😕",
    u"(?_?)":"😕",
    u">^_^<":"😃",
    u"<^!^>":"😃",
    u"^/^":"😃",
    u"（*^_^*）" :"😃",
    u"(^<^) (^.^)":"😃",
    u"(^^)":"😃",
    u"(^.^)":"😃",
    u"(^_^.)":"😃",
    u"(^_^)":"😃",
    u"(^^)":"😃",
    u"(^J^)":"😃",
    u"(*^.^*)":"😃",
    u"(^—^）":"😃",
    u"(#^.^#)":"😃",
    u"（^—^）":"👋",
    u"(;_;)/~~~":"👋",
    u"(^.^)/~~~":"👋",
    u"(-_-)/~~~ ($··)/~~~":"👋",
    u"(T_T)/~~~":"👋",
    u"(ToT)/~~~":"👋",
    u"(*^0^*)":"😍",
    u"(*_*)":"😍",
    u"(*_*;":"😍",
    u"(+_+) (@_@)":"😍",
    u"(*^^)v":"😂",
    u"(^_^)v":"😂",
    u'(-"-)':"😓",
    u"(ーー;)":"😓",
    u"(^0_0^)":"😎",
    u"(＾ｖ＾)":"😀",
    u"(＾ｕ＾)":"😀",
    u"(^)o(^)":"😀",
    u"(^O^)":"😀",
    u"(^o^)":"😀",
    u")^o^(":"😀",
    u":O o_O":"😮",
    u"o_0":"😮",
    u"o.O":"😮",
    u"(o.o)":"😮",
    u"oO":"😮",
}


def str2emoji(tweet):
	
	for pos,ej in enumerate(tweet):
		if ej in emojis:
			tweet[pos]=emojis[ej]
	return tweet

In [5]:
def norm_tweet(tweet):
  tweet = re.sub(r"\\u2019", "'", tweet)
  tweet = re.sub(r"\\u002c", ",", tweet)
  tweet=' '.join(str2emoji(unidecode(tweet).lower().split()))
  tweet = re.sub(r"\'ve", " have", tweet)
  tweet = re.sub(r" can\'t", " cannot", tweet)
  tweet = re.sub(r"n\'t", " not", tweet)
  tweet = re.sub(r"\'re", " are", tweet)
  tweet = re.sub(r"\'d", " would", tweet)
  tweet = re.sub(r"\'ll", " will", tweet)
  tweet = re.sub(r"\'s", "", tweet)
  tweet = re.sub(r"\'n", "", tweet)
  tweet = re.sub(r"\'m", " am", tweet)
  tweet = re.sub(r"@\w+", r' ',tweet)
  tweet = re.sub(r"#\w+", r' ',tweet)
  tweet = re.sub(r"[.]+"," ",tweet)
  #T = tokenizer.TweetTokenizer()
  tweet = T.tokenize(tweet)
  #tweet = [lemmatizer.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v']  else lemmatizer.lemmatize(i) for i,j in pos_tag(tknzr.tokenize(tweet))]
  tweet = [ i for i in tweet if (i not in stopwords) and (i not in string.punctuation ) ]
  #tweet = ' '.join(tweet)
  return tweet

In [6]:
def data_preprocessing(data):
  data['data'] = data['data'].apply(norm_tweet)
  if data['sentiment'].dtypes not in [np.int64]:
    data['sentiment']=data['sentiment'].apply(value_sentiment)
  return data['data'],data['sentiment'],data

In [7]:
import os.path
import scipy.stats
from sklearn import metrics
from sklearn.metrics import f1_score, jaccard_similarity_score

def cohen_kappa_score(y1, y2, labels=None, weights=None):
    """Cohen's kappa: a statistic that measures inter-annotator agreement.

    This function computes Cohen's kappa [1]_, a score that expresses the level
    of agreement between two annotators on a classification problem. It is
    defined as

    .. math::
        \kappa = (p_o - p_e) / (1 - p_e)

    where :math:`p_o` is the empirical probability of agreement on the label
    assigned to any sample (the observed agreement ratio), and :math:`p_e` is
    the expected agreement when both annotators assign labels randomly.
    :math:`p_e` is estimated using a per-annotator empirical prior over the
    class labels [2]_.

    Read more in the :ref:`User Guide <cohen_kappa>`.

    Parameters
    ----------
    y1 : array, shape = [n_samples]
        Labels assigned by the first annotator.

    y2 : array, shape = [n_samples]
        Labels assigned by the second annotator. The kappa statistic is
        symmetric, so swapping ``y1`` and ``y2`` doesn't change the value.

    labels : array, shape = [n_classes], optional
        List of labels to index the matrix. This may be used to select a
        subset of labels. If None, all labels that appear at least once in
        ``y1`` or ``y2`` are used.

    weights : str, optional
        List of weighting type to calculate the score. None means no weighted;
        "linear" means linear weighted; "quadratic" means quadratic weighted.

    Returns
    -------
    kappa : float
        The kappa statistic, which is a number between -1 and 1. The maximum
        value means complete agreement; zero or lower means chance agreement.

    References
    ----------
    .. [1] J. Cohen (1960). "A coefficient of agreement for nominal scales".
           Educational and Psychological Measurement 20(1):37-46.
           doi:10.1177/001316446002000104.
    .. [2] `R. Artstein and M. Poesio (2008). "Inter-coder agreement for
           computational linguistics". Computational Linguistics 34(4):555-596.
           <http://www.mitpressjournals.org/doi/abs/10.1162/coli.07-034-R2#.V0J1MJMrIWo>`_
    .. [3] `Wikipedia entry for the Cohen's kappa.
            <https://en.wikipedia.org/wiki/Cohen%27s_kappa>`_
    """
    confusion = metrics.confusion_matrix(y1, y2, labels=labels)
    n_classes = confusion.shape[0]
    sum0 = np.sum(confusion, axis=0)
    sum1 = np.sum(confusion, axis=1)
    expected = np.outer(sum0, sum1)*1.0 / np.sum(sum0)

    if weights is None:
        w_mat = np.ones([n_classes, n_classes], dtype=np.int)
        w_mat.flat[:: n_classes + 1] = 0
    elif weights == "linear" or weights == "quadratic":
        w_mat = np.zeros([n_classes, n_classes], dtype=np.int)
        w_mat += np.arange(n_classes)
        if weights == "linear":
            w_mat = np.abs(w_mat - w_mat.T)
        else:
            w_mat = (w_mat - w_mat.T) ** 2
    else:
        raise ValueError("Unknown kappa weighting type.")

    k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
    return 1 - k


def evaluate_ei(pred,gold):  
    """Calculates performance metrics for regression.
    
    :param pred: the file path of the predictions
    :param gold: the filte path withe gold data
    :return: a list with performace metrics.
    
    """
    
    f=open(pred, "rb")
    pred_lines=f.readlines()
    f.close()
    
    f=open(gold, "rb")
    gold_lines=f.readlines()
    f.close()    

    if(len(pred_lines)==len(gold_lines)):       
        # align tweets ids with gold scores and predictions
        data_dic={}


        header=True        
        for line in gold_lines:
            #line=line.decode('utf-8')
            
            if header:
                header=False
                continue

            parts=line.split('\t')
            if len(parts)==4:
                # tweet ids containing the word mystery are discarded
                if(not 'mystery' in parts[0]):
                    data_dic[parts[0]]=[float(line.split('\t')[3])]
            else:
                sys.exit('Format problem in '+os.path.basename(gold)+'. Please report this problem to the task organizers.')

        header=True        
        for line in pred_lines:
            
            if header:
                header=False
                continue

            parts=line.split('\t')      
            if len(parts)==4:
                # tweet ids containing the word mystery are discarded
                if(not 'mystery' in parts[0]):
                    if parts[0] in data_dic:
                        try:
                            data_dic[parts[0]].append(float(line.split('\t')[3]))
                        except ValueError:
                            # Invalid predictions are replaced by a default value
                            data_dic[parts[0]].append(0.5)
                    else:
                        sys.exit('Invalid tweet id ('+parts[0]+') in '+os.path.basename(pred)+'.')
            else:
                sys.exit('Format problem in '+os.path.basename(pred)+'.') 
            
        
        # lists storing gold and prediction scores
        gold_scores=[]  
        pred_scores=[]
         
        
        # lists storing gold and prediction scores where gold score >= 0.5
        gold_scores_range_05_1=[]
        pred_scores_range_05_1=[]
            
        for id in data_dic:
            if(len(data_dic[id])==2):
                gold_scores.append(data_dic[id][0])
                pred_scores.append(data_dic[id][1])
                
                if(data_dic[id][0]>=0.5):
                    gold_scores_range_05_1.append(data_dic[id][0])
                    pred_scores_range_05_1.append(data_dic[id][1])
                
            else:
                sys.exit('Repeated id ('+id+') in '+os.path.basename(pred)+' .')

                
                    
      
        # return zero correlation if predictions are constant
        if np.std(pred_scores)==0 or np.std(gold_scores)==0:
            return (0,0)
        

        pears_corr=scipy.stats.pearsonr(pred_scores,gold_scores)[0]                                                 
        pears_corr_range_05_1=scipy.stats.pearsonr(pred_scores_range_05_1,gold_scores_range_05_1)[0]                         
        return (pears_corr,pears_corr_range_05_1)                        
                          
    else:
        sys.exit('Predictions ('+os.path.basename(pred)+') and gold data ('+os.path.basename(gold)+') have different number of lines.')          
        
        
def evaluate_oc(pred,gold):  
    """Calculates performance metrics for ordinal classification.
    
    :param pred: the file path of the predictions
    :param gold: the filte path withe gold data
    :return: a list with performace metrics.
    
    """
  
   
    f=open(pred, "rb")
    pred_lines=f.readlines()
    f.close()
    
    f=open(gold, "rb")
    gold_lines=f.readlines()
    f.close()    


    if(len(pred_lines)==len(gold_lines)):       
        # align tweets ids with gold scores and predictions
        data_dic={}

    
        header=True        
        for line in gold_lines:
            
            if header:
                header=False
                continue
            
            parts=line.split('\t')
            
            label=int(parts[3].split(":")[0])
            
            if len(parts)==4:   
                data_dic[parts[0]]=[label]
                
            else:
                sys.exit('Format problem in '+os.path.basename(gold)+'. Please report this problem to the task organizers.')
        
        
        header=True         
        for line in pred_lines:
            if header:
                header=False
                continue            
            parts=line.split('\t')   
            label=int(parts[3].split(":")[0])
            if len(parts)==4:  
                if parts[0] in data_dic:
                    try:
                        data_dic[parts[0]].append(label)
                    except ValueError:
                        # Invalid predictions are replaced by a default value
                        data_dic[parts[0]].append(int(0))
                else:
                    sys.exit('Invalid tweet id ('+parts[0]+') in '+os.path.basename(pred)+'.')
            else:
                sys.exit('Format problem in '+os.path.basename(pred)+'.') 
            
        
        # lists storing gold and prediction scores
        gold_scores=[]  
        pred_scores=[]
         
        
        # lists storing gold and prediction scores where gold score >= 0.5
        gold_scores_some=[]
        pred_scores_some=[]
            
        for id in data_dic:
            if(len(data_dic[id])==2):
                gold_scores.append(data_dic[id][0])
                pred_scores.append(data_dic[id][1])
                
                if(data_dic[id][0]!=0):
                    gold_scores_some.append(data_dic[id][0])
                    pred_scores_some.append(data_dic[id][1])
                
            else:
                sys.exit('Repeated id ('+id+') in '+os.path.basename(pred)+' .')

                
                
        # return null scores if predictions are constant
        if np.std(pred_scores)==0 or np.std(gold_scores)==0:
            return (0,0,0,0)

        pears_corr=scipy.stats.pearsonr(pred_scores,gold_scores)[0]              
        pears_corr_some=scipy.stats.pearsonr(pred_scores_some,gold_scores_some)[0]  
        
        # fix labels to values observed in gold data        
        gold_labels=list(sorted(set(gold_scores)))
       
        kappa=cohen_kappa_score(pred_scores,gold_scores,labels=gold_labels, weights='quadratic')        
        kappa_some=cohen_kappa_score(pred_scores_some,gold_scores_some, labels=gold_labels, weights='quadratic')
 
        return (pears_corr,pears_corr_some,kappa,kappa_some)

    else:
        sys.exit('Predictions ('+os.path.basename(pred)+') and gold data ('+os.path.basename(gold)+') have different number of lines.')          
  
        
def evaluate_multilabel(pred,gold):  
    """Calculates performance metrics for multi-label classification.
    
    :param pred: the file path of the predictions
    :param gold: the filte path withe gold data
    :return: a list with performace metrics.    
    """     
    
    f=open(pred, "rb")
    pred_lines=f.readlines()
    f.close()
    
    f=open(gold, "rb")
    gold_lines=f.readlines()
    f.close()    

    if(len(pred_lines)==len(gold_lines)):       
        # align tweets ids with gold scores and predictions
        data_dic={}

    
        header=True        
        for line in gold_lines:
           
            if header:
                header=False
                continue
            
            parts=line.split('\t')
            
            if len(parts)==13:  
                labels=[]
                for m_label in parts[2:13]:
                    labels.append(int(m_label))
 
                data_dic[parts[0]]=[tuple(labels)]
                
            else:
                sys.exit('Format problem in '+os.path.basename(gold)+'. Please report this problem to the task organizers.')
        
        header=True         
        for line in pred_lines:
            if header:
                header=False
                continue            
            parts=line.split('\t')   
            if len(parts)==13:  
                if parts[0] in data_dic:
                    try:
                        labels=[]
                        for m_label in parts[2:13]:
                            labels.append(int(m_label))
                        data_dic[parts[0]].append(tuple(labels))
                    except ValueError:
                        # Invalid predictions are replaced by a default value
                        data_dic[parts[0]].append((0,0,0,0,0,0,0,0,0,0,0))
                else:
                    sys.exit('Invalid tweet id in '+os.path.basename(pred)+'.')
            else:
                sys.exit('Format problem in '+os.path.basename(pred)+'.')   
            

       # lists storing gold and prediction scores
        gold_scores=[]  
        pred_scores=[]
         
            
        for id in data_dic:
            if(len(data_dic[id])==2):
                gold_scores.append(data_dic[id][0])
                pred_scores.append(data_dic[id][1])
                
            else:
                sys.exit('Repeated id ('+id+') in '+os.path.basename(pred)+' .')


        y_true = np.array(gold_scores)
        y_pred = np.array(pred_scores)       
    
        acc=jaccard_similarity_score(y_true,y_pred)       
        f1_micro=f1_score(y_true, y_pred, average='micro')  
        f1_macro=f1_score(y_true, y_pred, average='macro')  
            
        return (acc,f1_micro,f1_macro)
    
    else:
        sys.exit('Predictions ('+os.path.basename(pred)+') and gold data ('+os.path.basename(gold)+') have different number of lines.')          

In [8]:
def model1(x_train_3, y_train_3,x_val_3, y_val_3, embedding_layer):

	model1 = Sequential()
	model1.add(embedding_layer)
	model1.add(LSTM(32))
	model1.add(Dropout(0.2))
	model1.add(Dense(32, activation='relu'))
	model1.add(Dropout(0.2))
	model1.add(Dense(3, activation='softmax'))
	model1.compile(loss='categorical_crossentropy',
			      optimizer='Adam',
			      metrics=['acc'])
	model1.summary()
	history=model1.fit(x_train_3, y_train_3, validation_data=(x_val_3, y_val_3),epochs=6, batch_size=50)
	model1.save("./model1.h5")

In [9]:
def model2(x_train_3, y_train_3,x_val_3, y_val_3, embedding_layer,epochs, batch_size):
	model2 = Sequential()
	model2.add(embedding_layer)
	model2.add(GRU(32))
	model2.add(Dropout(0.2))
	model2.add(Dense(3, activation='softmax'))
	model2.compile(loss='categorical_crossentropy',
			      optimizer='rmsprop',
			      metrics=['acc'])
	model2.summary()
	history=model2.fit(x_train_3, y_train_3, validation_data=(x_val_3, y_val_3),epochs=epochs, batch_size=batch_size)
	model2.save("./model2.h5")

In [10]:
#Path of datas
data_train_3 =  "train_1_1.csv"
data_train_7 =  "train_3_3.csv"

In [11]:
#Load Data
data_train_3= pd.read_csv(data_train_3,sep='\t',names=['id','sentiment','data'])
data_train_7= pd.read_csv(data_train_7,sep='\t',names=['id','sentiment','data'])

In [12]:
# Apply the preprocessiog on the data
# Normalize the tweets
tweets_train_3, sentiments_train_3,data_train_3 = data_preprocessing(data_train_3)
tweets_train_7, sentiments_train_7,data_train_7 = data_preprocessing(data_train_7)

In [13]:
all_tweet = tweets_train_3.append(tweets_train_7)
tokenizer = Tokenizer(filters=' ')
tokenizer.fit_on_texts(all_tweet)
word_index = tokenizer.word_index

In [14]:
sequences_train_3 = tokenizer.texts_to_sequences(tweets_train_3)
sequences_train_7 = tokenizer.texts_to_sequences(tweets_train_7)
sequences = sequences_train_3 + sequences_train_7


In [15]:
#Seek the longuest Tweet
#To Apply a Padding
MAX_SEQUENCE_LENGTH = 0
for elt in sequences:
	if len(elt) > MAX_SEQUENCE_LENGTH:
		MAX_SEQUENCE_LENGTH = len(elt)
  
data_train_3 = pad_sequences(sequences_train_3, maxlen=MAX_SEQUENCE_LENGTH)
data_train_7 = pad_sequences(sequences_train_7, maxlen=MAX_SEQUENCE_LENGTH)

In [16]:
indices_train_3 = np.arange(data_train_3.shape[0])
data_train_3 = data_train_3[indices_train_3]

indices_train_7 = np.arange(data_train_7.shape[0])
data_train_7 = data_train_7[indices_train_7]

labels_train_3 = to_categorical(np.asarray(sentiments_train_3), 3)
labels_train_3 = labels_train_3[indices_train_3]

In [17]:
nb_words=len(word_index)+1
EMBEDDING_DIM=300
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [18]:
oov=[]
oov.append((np.random.rand(EMBEDDING_DIM)) - 1.0)
oov = oov / np.linalg.norm(oov)

In [19]:
for word, i in word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)
    else:
        embedding_matrix[i] = oov

In [20]:
split_idx = int(len(data_train_3)*0.70)
x_train_3, x_val_3 = data_train_3[:split_idx], data_train_3[split_idx:]
y_train_3, y_val_3 = labels_train_3 [:split_idx], labels_train_3[split_idx:]

In [21]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False, name='embedding_layer')

In [22]:
model1(x_train_3, y_train_3,x_val_3, y_val_3, embedding_layer)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 31, 300)           18438300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 18,482,079
Trainable params: 43,779
Non-trainable params: 18,438,300
_____________________________________

In [23]:
model2(x_train_3, y_train_3,x_val_3, y_val_3, embedding_layer, 6, 50)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 31, 300)           18438300  
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                31968     
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 18,470,367
Trainable params: 32,067
Non-trainable params: 18,438,300
_________________________________________________________________
Train on 35233 samples, validate on 15100 samples
Epoch 1/6
35233/35233 [==============================] - 9s 259us/step - loss: 0.8336 - acc: 0.6113 - val_loss: 0.8331 - val_acc: 0.5971
Epoch 2/6
35233/35233 [=========================

In [24]:
print(sentiments_train_7, tweets_train_7)
labels_train_7 = to_categorical(np.asarray(sentiments_train_7), 7)
labels_train_7 = labels_train_7[indices_train_7]

split_idx = int(len(data_train_7)*0.85)
x_train_7, x_val_7 = data_train_7[:split_idx], data_train_7[split_idx:]
y_train_7, y_val_7 = labels_train_7 [:split_idx], labels_train_7[split_idx:]

0       0
1       0
2       0
3       0
4      -2
       ..
1625    3
1626   -2
1627    1
1628    0
1629   -2
Name: sentiment, Length: 1630, dtype: int64 0                             [yeah, ☺, ️, playing, well]
1       [least, guy, trying, discourage, anymore, want...
2       [uplift, still, discouraged, means, listening,...
3                              [age, heyday, blood, tame]
4       [embarrassed, saw, us, like, knvfkkjg, thinks,...
                              ...                        
1625       [idk, help, someone, smile, comes, lips, n, n]
1626    [think, leep, favorite, get, dark, maybe, inso...
1627                   [amelia, want, sarah, v, grateful]
1628                                  [lack, makes, n, n]
1629                   [james, clapper, cary, disturbing]
Name: data, Length: 1630, dtype: object


In [25]:
model=keras.models.load_model("./model1.h5")
model.summary()
model.layers.pop()
model.layers.pop()
model.add(Dense(150,activation='relu',name='dense1'))
model.add(Dense(64,activation='relu',name='dense2'))
model.add(Dense(7,activation='softmax',name='dense3'))
model.summary()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["accuracy"])
history = model.fit(x_train_7, y_train_7,   validation_data=(x_val_7,y_val_7), epochs=11, batch_size=50)
model.save("./model3.h5")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 31, 300)           18438300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 18,482,079
Trainable params: 43,779
Non-trainable params: 18,438,300
_____________________________________

/home/beuvry_j/.local/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Train on 1385 samples, validate on 245 samples
Epoch 1/11
1385/1385 [==============================] - 1s 556us/step - loss: 1.8274 - accuracy: 0.2939 - val_loss: 1.7912 - val_accuracy: 0.2857
Epoch 2/11
1385/1385 [==============================] - 0s 275us/step - loss: 1.6932 - accuracy: 0.3242 - val_loss: 1.7084 - val_accuracy: 0.2694
Epoch 3/11
1385/1385 [==============================] - 0s 280us/step - loss: 1.6172 - accuracy: 0.3487 - val_loss: 1.6664 - val_accuracy: 0.3184
Epoch 4/11
1385/1385 [==============================] - 0s 283us/step - loss: 1.5726 - accuracy: 0.3502 - val_loss: 1.6690 - val_accuracy: 0.3347
Epoch 5/11
1385/1385 [==============================] - 0s 286us/step - loss: 1.5450 - accuracy: 0.3545 - val_loss: 1.6587 - val_accuracy: 0.2980
Epoch 6/11
1385/1385 [==============================] - 0s 280us/step - loss: 1.5249 - accuracy: 0.3863 - val_loss: 1.6626 - val_accuracy: 0.3061
Epoch 7/11
1385/1385 [==============================] - 0s 280us/step - loss:

# Testing

In [46]:
# Load the data
corpora_test_7="./train_3_3.csv"
corpora_test_7= pd.read_csv(corpora_test_7,sep='\t',names=['id','sentiment','data'])

tweets_test_7,sentiments_test_7,corpora_test_processed_7 = data_preprocessing(corpora_test_7.copy())
sequences_test_7 = tokenizer.texts_to_sequences(tweets_test_7)
pad_sequence_test_7 = pad_sequences(sequences_test_7, maxlen=MAX_SEQUENCE_LENGTH)

In [41]:
# Predict the classes
y_test_predict_7 = model.predict_classes(pad_sequence_test_7) - 3 # To replace between -3 and 3
y_test_predict_7_classes = np.vectorize(lambda index: "{}: ".format(index) + {
    -3:"very negative",
    -2:"moderately negative",
    -1:"slightly negative",
    0:"neutral or mixed",
    1:"slightly positive",
    2:"moderately positive",
    3:"very positive" }[index] + " emotional state can be inferred")(y_test_predict_7)

#cohen_kappa_score(y_test_predict_7, sentiments_test_7.values)
y_test_predict_7, y_test_predict_7_classes

(array([ 0, -3, -3, ..., -3,  2,  2]),
 array(['0: neutral or mixed emotional state can be inferred',
        '-3: very negative emotional state can be inferred',
        '-3: very negative emotional state can be inferred', ...,
        '-3: very negative emotional state can be inferred',
        '2: moderately positive emotional state can be inferred',
        '2: moderately positive emotional state can be inferred'],
       dtype='<U55'))

In [48]:
# Output to file

output_df = corpora_test_7.drop(columns=["sentiment"]).rename(columns = { "id": "ID", "data": "Tweet" })
output_df["Affect Dimension"] = "valence"
output_df["Intensity Class"] = y_test_predict_7_classes

output_df.to_csv("result_TP1.csv", sep="\t", index=False)

output_df

,ID,Tweet,Affect Dimension,Intensity Class
0,0,@liamch88 yeah! :) playing well,valence,0: neutral or mixed emotional state can be inf...
1,1,At least I don't have a guy trying to discoura...,valence,-3: very negative emotional state can be inferred
2,2,UPLIFT: If you're still discouraged it means y...,valence,-3: very negative emotional state can be inferred
3,3,"...at your age, the heyday in the blood is tam...",valence,-3: very negative emotional state can be inferred
4,4,i was so embarrassed when she saw us i was lik...,valence,2: moderately positive emotional state can be ...
...,...,...,...,...
1625,1625,"Idk why but when I help someone, a smile come...",valence,-3: very negative emotional state can be inferred
1626,1626,I think 'Sleep' is my favorite from How did we...,valence,-3: very negative emotional state can be inferred
1627,1627,What does Amelia want?! Sarah was v grateful ...,valence,-3: very negative emotional state can be inferred
1628,1628,It’s lack of #faith that makes #people #afraid...,valence,2: moderately positive emotional state can be ...
